In [ ]:
# packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os, os.path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import xlwings as xw
import time

In [ ]:
from ipynb.fs.full.f_newbuild import calc_new_build
from ipynb.fs.full.f_pv22to14 import convert_pv_jobs
from ipynb.fs.full.f_transmission import calculate_transmission_jobs
from ipynb.fs.full.f_fuel_trade import fuel_trade_jobs
from ipynb.fs.full.f_nameplate import calc_nameplate

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def ReEDS_to_Jobs(analysis,capfac_scenarios,tech_nuclear,sc):
    
    """
    analysis = "v6.2"
    capfac_scenarios = "low" # low or mid or high
    tech_nuclear = "jedi" # "nei" will replace the jedi data, the other option is "jedi"
    sc="Mid_Case"
    """
    
    start_time = time.time()
    scenario = "StdScen21_"+sc+"_annual_state"
    
    elec_path = r"../../ReEDS_StandardScenario/"+scenario+".csv"
    elec_data = pd.read_csv(elec_path,skiprows = 3)
    elec_data = elec_data.set_index('t')

    all_states = elec_data["state"].unique()
    all_year = elec_data.index.unique()
    state_abbrs = pd.read_excel(r"../Census/State_Abbr.xlsx")
    start_data = pd.read_excel("base_data.xlsx")
    state_abbrs.head()

    # isolate the generation data 
    data_MWh = elec_data.filter(regex='MWh')
    data_MWh = data_MWh.drop(columns="generation_MWh")
    data_MWh["state"] = elec_data["state"]

    # isolate the capacity data
    data_MW = elec_data.filter(regex='MW')
    cols_without_MWh = [x for x in list(data_MW.columns) if 'MWh' not in x]
    data_MW = data_MW[cols_without_MWh]
    data_MW["state"] = elec_data["state"]

    # match ReEDS technologies with technologies available to JEDI
    tech_spec = pd.read_excel("tech_specs.xlsx",sheet_name="main")
    jedi_sheets = pd.read_excel("tech_specs.xlsx",sheet_name="jedi_tech")
    tech_spec = pd.merge(tech_spec,jedi_sheets,left_on="jedi_fname", right_on="jedi_fname")

    # standard ReEDS facility build rate
    build_rate_all = pd.read_csv(r"../../ReEDS_OpenAccess-main/inputs/construction_times/construction_times_default.csv")
    build_rate_rename = pd.read_excel("ReEDS_construction_rename.xlsx",sheet_name="main")

    # nameplate capacities
    all_nameplate_scenarios = pd.read_csv(r"../Nameplate_Capacity/all_cap_fac_scenarios.csv")

    # check which generation technologies exist across the board
    check_data = elec_data.loc[:, (elec_data != 0).any(axis=0)]
    check_data_MWh = check_data.filter(regex='MWh')
    check_data_MWh = check_data_MWh.drop(columns="generation_MWh")

    # calculate the new builds
    from ipynb.fs.full.f_newbuild import calc_new_build
    all_newbuild = calc_new_build(all_states,data_MW)

    # fuel local shares
    coal_share = pd.read_excel(r"../Fuel_production/localshare_coal_state.xlsx")
    ng_share = pd.read_excel(r"../Fuel_production/localshare_ng_state.xlsx")
    oil_share = pd.read_csv(r"../Fuel_production/Crude_Oil_Production_2020.csv")

    # fuel price projection
    fuel_prices = pd.read_csv("Table_3._Energy_Prices_by_Sector_and_Source.csv",skiprows=4)
    fuel_prices = fuel_prices.rename(columns={"Unnamed: 0":"Name"})
    fuel_prices_ur = fuel_prices.query("Name=='Uranium'").query("units=='2019 $/MMBtu'")

    fuel_prices_dist_oil = fuel_prices.query("`full name` =='Energy Prices: Electric Power: Distillate Fuel Oil: Reference case'")
    fuel_prices_res_oil = fuel_prices.query("`full name` =='Energy Prices: Electric Power: Residual Fuel Oil: Reference case'")

    # JOB MODEL VARIATION - nuclear technology 
    nuclear_jobs = pd.read_excel("test_nuclear.xlsx")
    nuclear_list = nuclear_jobs['Connecticut'].tolist()
    
    tech_ind = 0
    trade_ind = 0
    app = xw.apps.active
    for elec_col in check_data_MWh.columns:
        this_tech = elec_col[:-4]
        # skip imports as it doesn't generate jobs 
        # skipping some other technologies
        if this_tech in ["imports"]:
            continue
        br_name = build_rate_rename.query("i == @this_tech")["ref"].iloc[0]

        # type of technology for different ways to assign input data
        tech_type = tech_spec.query("tech == @this_tech")["type"].iloc[0]
        tech_name = tech_spec.query("tech == @this_tech")["jedi_fname"].iloc[0]
        tech_path = r"../../JEDI/"+tech_name+".xlsm"
        # open the workbook
        wb = xw.Book(tech_path)
        sheet = wb.sheets['ProjectData']

        state_ind = 0
        for this_state in list(all_states):

            ############ isolate the ReEDS capacity and generation (technology, state)
            this_state_data_MW = data_MW.query('state == @this_state')
            this_state_data_MWh = data_MWh.query('state == @this_state')
            this_state_newbuild = all_newbuild.query('state == @this_state')
            this_state_data_MW = this_state_data_MW.loc[:, (this_state_data_MW != 0).any(axis=0)]
            this_state_data_MWh = this_state_data_MWh.loc[:, (this_state_data_MWh != 0).any(axis=0)]
            this_state_newbuild = this_state_newbuild.loc[:, (this_state_newbuild != 0).any(axis=0)]
            # if this technology doesn't exist on the list of new builds and generation, skip this round 
            # what passes will have either generation data, new build data, or both
            if this_tech+"_MWh" not in this_state_data_MWh.columns and this_tech+"_MW" not in this_state_newbuild.columns:
                continue

            # assign the state to the project description 
            this_state_full = state_abbrs.query("Abbr == @this_state")["Caps"].iloc[0]
            this_state_lower = state_abbrs.query("Abbr == @this_state")["State"].iloc[0]
            sheet.range(tech_spec.query("tech == @this_tech")["state"].iloc[0]).value = this_state_full # state

            om_data = start_data.copy()
            const_data = start_data.iloc[:-1,:].copy()
            ############ isolate the annual generation (technology, state, year)
            for this_year in list(all_year):
                this_ind = list(all_year).index(this_year)
                # find the build year based on the ReEDS database
                if this_tech == "dac": # bypass the dac build year of 1 to Climeworks and Carbon Engineering's numbers 
                    build_year = 2
                else:
                    build_year = build_rate_all.query("i ==@br_name").query("t_online ==@this_year")["construction_time"].iloc[0]


                ############ JOBS ############
                # calculate the capacity factor based on the generation and capacity of that year
                if this_tech+"_MWh" in this_state_data_MWh.columns:
                    cap_fac = this_state_data_MWh[this_tech+"_MWh"].iloc[this_ind]/(this_state_data_MW[this_tech+"_MW"].iloc[this_ind]*365*24)
                    if this_tech == "dac":
                        cap_fac = -cap_fac # since MWh is negative, to make the cap fac positive
                # select the number of new builds
                if this_tech+"_MW" in this_state_newbuild.columns:
                    new_cap = this_state_newbuild[this_tech+"_MW"].iloc[this_ind]
                # calculate the construction year
                const_year = this_year - build_year # the year of construction 
                # JEDI sets the maximum limit of the construction year to be 2030
                # if the construction year is beyond 2030, assume it to be 2030
                if const_year >= 2030:
                    const_year = 2030
                sheet.range(tech_spec.query("tech == @this_tech")["const_year"].iloc[0]).value = const_year # construction start year

                ############ vary other project definitions based on technology types
                if tech_type == "thermal" or tech_type == "hydro": 
                    sheet.range(tech_spec.query("tech == @this_tech")["capacity_fac"].iloc[0]).value = cap_fac # capacity factor
                    if tech_name == "01d-jedi-coal-model-rel-c12-23-16":
                        state_coal_local = coal_share.query("Abbr == @this_state").iloc[0,-1]/100 # Coal local share
                        sheet.range(tech_spec.query("tech == @this_tech")["user_def"].iloc[0]).value = state_coal_local
                    elif tech_name == "01d-jedi-ngas-model_rel-ng4-17-17" or this_tech == "ng-cc-ccs":
                        state_ng_local = coal_share.query("Abbr == @this_state").iloc[0,-1]/100 # Natural gas local share 
                        sheet.range(tech_spec.query("tech == @this_tech")["user_def"].iloc[0]).value = state_ng_local
                    elif tech_name == "01d-jedi-nuclear-usermodifed": # insert Uranium price
                        fuel_year = str(this_year)
                        ur_price = fuel_prices_ur[fuel_year].iloc[0]
                        sheet.range(tech_spec.query("tech == @this_tech")["user_def"].iloc[0]).value = ur_price
                    elif this_tech == "oil-gas-steam": 
                        fuel_year = str(this_year) # insert oil price and convert to 2011 dollars (based on the model)
                        oil_price = (fuel_prices_dist_oil[fuel_year].iloc[0] + fuel_prices_res_oil[fuel_year].iloc[0])/2/1.14 
                        sheet.range(tech_spec.query("tech == @this_tech")["user_def"].iloc[0]).value = oil_price
                        state_oil_local = oil_share.query("Abbr == @this_state").iloc[0,-1]/100 # Oil local share 
                        sheet.range(tech_spec.query("tech == @this_tech")["user_def2"].iloc[0]).value = state_oil_local
                elif tech_type == "pv":
                    if this_tech == "rooftop pv":
                        sheet.range(tech_spec.query("tech == @this_tech")["user_def"].iloc[0]).value = "Residential"
                    elif this_tech == "utility pv":
                        sheet.range(tech_spec.query("tech == @this_tech")["user_def"].iloc[0]).value = "Utility"

                # vary nameplate capacity, except for user modified models which can't
                if this_tech not in ["battery-2h","battery-4h","battery-6h","battery-8h","battery-10h","biopower ccs",
                                    "dac","h2-ct","ng-cc-ccs","nuclear","oil-gas-steam","pv+battery"]:
                    #hist_nameplate = calc_nameplate(this_state,this_tech,capfac_scenarios)
                    hist_nameplate = all_nameplate_scenarios.query("state==@this_state").query("tech==@this_tech").query("scenario==@capfac_scenarios")["value"].iloc[0]
                    sheet.range(tech_spec.query("tech == @this_tech")["nameplate"].iloc[0]).value = hist_nameplate

                ############ calculate the number of multiples of facilities with such nameplate capacity 
                # read the nameplate capacity 
                name_plate = sheet.range(tech_spec.query("tech == @this_tech")["nameplate"].iloc[0]).value 
                if tech_type == "pv" or tech_type == "battery":
                    name_plate = name_plate/1000 # pv nameplate capacity is in kW

                ############ find the number of jobs under JEDI categories 
                # click one button for offshore wind
                if this_tech == "offshore wind":
                    wb.macro('elec_recalc')()
                # remove None types
                sheet3 = wb.sheets["Calculations"]

                # generation jobs need to be in the dataframe column, otherwise automatically set to zero
                if this_tech+"_MWh" in this_state_data_MWh.columns:
                    if this_state_data_MWh[this_tech+"_MWh"].iloc[this_ind] != 0:
                        jobs_omdir = np.asarray(sheet3.range(tech_spec.query("tech == @this_tech")["annualOM_wPlantEmployee_direct"].iloc[0]).value)
                        jobs_omindir = np.asarray(sheet3.range(tech_spec.query("tech == @this_tech")["annualOM_wPlantEmployee_indirect"].iloc[0]).value)
                        jobs_omdir = [0 if v is None else v for v in jobs_omdir]
                        jobs_omindir = [0 if v is None else v for v in jobs_omindir]
                        total_cap = this_state_data_MW[this_tech+"_MW"].iloc[this_ind]
                        num_nameplates = total_cap/name_plate # number of multiples for operation
                        if this_tech == "dac": # since we don't want to vary the capacity factor 
                            jobs_om = np.asarray(np.add(jobs_omdir,jobs_omindir))*num_nameplates*cap_fac
                        else:
                            jobs_om = np.asarray(np.add(jobs_omdir,jobs_omindir))*num_nameplates
                    else:
                        jobs_om = [0] *15
                else:
                    jobs_om = [0] *15
                # construction jobs need to be in the dataframe column, otherwise automatically set to zero
                if this_tech+"_MW" in this_state_newbuild.columns:
                    if this_state_newbuild[this_tech+"_MW"].iloc[this_ind] != 0:
                        jobs_constdir = np.asarray(sheet3.range(tech_spec.query("tech == @this_tech")["totCIM_direct"].iloc[0]).value)
                        jobs_constindir = np.asarray(sheet3.range(tech_spec.query("tech == @this_tech")["totCIM_indirect"].iloc[0]).value)
                        jobs_constdir = [0 if v is None else v for v in jobs_constdir]
                        jobs_constindir = [0 if v is None else v for v in jobs_constindir]
                        num_newbuild = new_cap/name_plate # number of multiples for new build
                        jobs_const = np.asarray(np.add(jobs_constdir,jobs_constindir))*num_newbuild
                    else:
                        jobs_const = [0] * 14
                else:
                    jobs_const = [0] * 14

                # convert the 22 pv industries into 14 JEDI industries 
                if tech_type == "pv" or tech_type == "battery":
                    if len(jobs_om) != 15:
                        jobs_om = convert_pv_jobs(jobs_om)
                    if len(jobs_const) != 14:
                        jobs_const = convert_pv_jobs(jobs_const)

                # manually add mining jobs - both direct and indirect
                if tech_name == "01d-jedi-coal-model-rel-c12-23-16" and this_tech+"_MWh" in this_state_data_MWh:
                    jobs_om[1] = this_state_data_MWh[this_tech+"_MWh"].iloc[this_ind] * (4.374/100000) * state_coal_local

                # TEST NUCLEAR
                if this_tech == "nuclear" and "nuclear_MWh" in this_state_data_MWh.columns and tech_nuclear =="nei":
                    jobs_om = np.asarray(nuclear_list)*total_cap

                # assign the number of jobs under this technology this year to the dataframe 
                om_data[this_year] = jobs_om
                const_data[this_year] = jobs_const

                # FUEL TRADE JOBS calculated separated
                if this_tech == "coal" or this_tech == "ng-cc" or this_tech == "ng-ct" or this_tech == "ng-cc-ccs":
                    if this_tech+"_MWh" in this_state_data_MWh:
                        outsourced = fuel_trade_jobs(this_state,this_tech,this_state_data_MWh[this_tech+"_MWh"].iloc[this_ind],this_year) # when zero, it means not on trade database
                        if trade_ind == 0 and isinstance(outsourced, pd.DataFrame): 
                            trade_jobs = outsourced.copy()
                            trade_ind = trade_ind+1
                        elif trade_ind != 0 and isinstance(outsourced, pd.DataFrame):
                            trade_jobs = trade_jobs.append(outsourced)
                            trade_ind = trade_ind+1 # only counts if entered this loop

            # fill in the gaps of years without data to be the average of the year before and the year after 
            this_state_om = om_data.copy().set_index("i")
            for y in this_state_om.columns:
                if np.isnan(this_state_om[y].iloc[-1]):
                    this_state_om[y] = (this_state_om[y-1]+this_state_om[y+1])/2
            # assume construction jobs to average over construction period 
            this_state_const_raw = const_data.copy().set_index("i")
            this_state_const = this_state_const_raw.copy()
            this_state_const.loc[:,:] = 0 # set the whole sheet to zero
            # add everything on top of the empty sheet
            for y in this_state_const.columns:
                if np.isnan(this_state_const_raw[y].iloc[0]) == False and build_year>1:
                    for t in np.arange(build_year):
                        if y-t > 2021:
                            this_state_const[y-t] = this_state_const[y-t]+this_state_const_raw[y]/build_year
            this_state_const = this_state_const.replace(np.nan,0)

            # change the format of this data to fit the JEDI_output format
            this_state_om_t = this_state_om.transpose()
            this_state_const_t = this_state_const.transpose()
            # assign the jobs under "plant employees" to the utilities or TCPU (generation only)
            this_state_om_t["TCPU"] = this_state_om_t["TCPU"] + this_state_om_t["Plant Employees"]
            this_state_om_t = this_state_om_t.drop(columns = "Plant Employees")
            # assign the state and technology to the output data 
            this_state_om_t["state"] = this_state
            this_state_om_t["tech"] = this_tech
            this_state_om_t["type"] = "OM"
            this_state_const_t["state"] = this_state
            this_state_const_t["tech"] = this_tech
            this_state_const_t["type"] = "CIM"

            # append data from each state
            if state_ind == 0:
                JEDI_output_state = this_state_om_t.copy()
                JEDI_output_state = JEDI_output_state.append(this_state_const_t)
            else:
                JEDI_output_state = JEDI_output_state.append(this_state_om_t)
                JEDI_output_state = JEDI_output_state.append(this_state_const_t)
            state_ind = state_ind + 1

        # close the workbook
        wb.close()

        # append data from each tech ###acually tech here
        if tech_ind == 0:
            JEDI_output = JEDI_output_state
        else:
            JEDI_output = JEDI_output.append(JEDI_output_state)
        tech_ind = tech_ind+1

    # add transmission jobs
    transmission_jobs = calculate_transmission_jobs(sc)
    JEDI_output_end = JEDI_output.append(transmission_jobs)

    # add fuel trade jobs
    trade_jobs_final = trade_jobs.copy().groupby(["state","year","tech","type"]).sum()
    trade_jobs_final = trade_jobs_final.reset_index().set_index("year")
    trade_jobs_final["TCPU"] = trade_jobs_final["TCPU"]+trade_jobs_final["Plant Employees"]
    trade_jobs_final = trade_jobs_final.drop(columns="Plant Employees")
    JEDI_output_end = JEDI_output_end.append(trade_jobs_final)
    
    JEDI_output_end.to_csv(r"output/JEDI_output_"+scenario+"_"+analysis+tech_nuclear+"_"+capfac_scenarios+"_both.csv")
    end_time = time.time()
    time_to_run = str(round((end_time - start_time)/60,2))
    
    return time_to_run